In [1]:
import pandas as pd

In [2]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import random, pickle
from collections import Counter
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [8]:
df=pd.read_csv('/content/IMDB Dataset.csv',encoding='latin-1', on_bad_lines='skip')

In [9]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [10]:
df.shape

(50000, 2)

In [11]:
y=df['sentiment']

In [12]:
y

,sentiment
0,positive
1,positive
2,positive
3,negative
4,positive
...,...
49995,positive
49996,negative
49997,negative
49998,negative


In [13]:
x=df['review']

In [18]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Initialize stop words and lemmatizer once
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Function to tokenize and remove stop words for a single review
def tokenize_and_filter(review_text):
    lowercased_review = review_text.lower()
    words = word_tokenize(lowercased_review)
    filtered_words = [word for word in words if word not in stop_words]
    return filtered_words

# Apply this function to each review in the 'review' column ('x')
df['tokenized_review'] = x.apply(tokenize_and_filter)

In [17]:
df['tokenized_review'].head()

,tokenized_review
0,"[one, reviewers, mentioned, watching, 1, oz, e..."
1,"[wonderful, little, production, ., <, br, /, >..."
2,"[thought, wonderful, way, spend, time, hot, su..."
3,"[basically, 's, family, little, boy, (, jake, ..."
4,"[petter, mattei, 's, ``, love, time, money, ''..."


In [19]:
def lemmatize_words(word_list):
    return [lemmatizer.lemmatize(word) for word in word_list]

df['lemmatized_review'] = df['tokenized_review'].apply(lemmatize_words)

df['lemmatized_review'].head()

,lemmatized_review
0,"[one, reviewer, mentioned, watching, 1, oz, ep..."
1,"[wonderful, little, production, ., <, br, /, >..."
2,"[thought, wonderful, way, spend, time, hot, su..."
3,"[basically, 's, family, little, boy, (, jake, ..."
4,"[petter, mattei, 's, ``, love, time, money, ''..."


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Join the lemmatized words back into a single string for TfidfVectorizer
df['lemmatized_review_str'] = df['lemmatized_review'].apply(lambda x: ' '.join(x))

# Initialize TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000) # Limiting to 5000 features for practical reasons

# Fit and transform the lemmatized reviews
X = tfidf_vectorizer.fit_transform(df['lemmatized_review_str'])

print(f"Shape of the vectorized data (X): {X.shape}")

Shape of the vectorized data (X): (50000, 5000)


In [24]:
X[0]

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 114 stored elements and shape (1, 5000)>

# Task
Perform sentiment classification on the prepared TF-IDF vectorized text data (`X`) and sentiment labels (`y`). First, split the data into training and testing sets. Then, train and evaluate multiple classification models, including Logistic Regression, Multinomial Naive Bayes, and Linear SVC, displaying a classification report for each. Finally, identify the best-performing model based on the evaluation metrics and present its classification report, summarizing the overall findings.

## Split Data

### Subtask:
Split the vectorized data (X) and the sentiment labels (y) into training and testing sets using `train_test_split` from `sklearn.model_selection`.


**Reasoning**:
First, import the `train_test_split` function from `sklearn.model_selection`, then split the data into training and testing sets using the specified parameters.



In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of y_test: {y_test.shape}")

Shape of X_train: (40000, 5000)
Shape of X_test: (10000, 5000)
Shape of y_train: (40000,)
Shape of y_test: (10000,)


## Train and Evaluate Models

### Subtask:
Train multiple classification models (e.g., Logistic Regression, Multinomial Naive Bayes, Linear SVC) from scikit-learn on the training data. After training, evaluate each model's performance on the test data and generate a classification report for each model.


**Reasoning**:
To begin training and evaluating the classification models, I need to import the required model classes and the `classification_report` function from their respective scikit-learn modules. This is the first instruction provided in the subtask.



In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

# --- Logistic Regression ---
print("\n--- Logistic Regression ---")
log_reg_model = LogisticRegression(max_iter=1000) # Increased max_iter for convergence
log_reg_model.fit(X_train, y_train)
y_pred_log_reg = log_reg_model.predict(X_test)
print("Classification Report for Logistic Regression:")
print(classification_report(y_test, y_pred_log_reg))

# --- Multinomial Naive Bayes ---
print("\n--- Multinomial Naive Bayes ---")
mnb_model = MultinomialNB()
mnb_model.fit(X_train, y_train)
y_pred_mnb = mnb_model.predict(X_test)
print("Classification Report for Multinomial Naive Bayes:")
print(classification_report(y_test, y_pred_mnb))

# --- Linear SVC ---
print("\n--- Linear SVC ---")
svc_model = LinearSVC(max_iter=10000) # Increased max_iter for convergence
svc_model.fit(X_train, y_train)
y_pred_svc = svc_model.predict(X_test)
print("Classification Report for Linear SVC:")
print(classification_report(y_test, y_pred_svc))


--- Logistic Regression ---
Classification Report for Logistic Regression:
              precision    recall  f1-score   support

    negative       0.90      0.87      0.89      4961
    positive       0.88      0.90      0.89      5039

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000


--- Multinomial Naive Bayes ---
Classification Report for Multinomial Naive Bayes:
              precision    recall  f1-score   support

    negative       0.86      0.85      0.85      4961
    positive       0.85      0.86      0.86      5039

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000


--- Linear SVC ---
Classification Report for Linear SVC:
              precision    recall  f1-score   support

    negative       0.89      0.87      0.88      4961
    positive       

## Identify Best Model and Report

### Subtask:
Compare the performance metrics (e.g., accuracy, F1-score) of all trained models to identify the best one. Then, display the classification report for the best performing model.


### Identify Best Model and Report

Based on the classification reports from the previous step:
- **Logistic Regression**: Accuracy 0.89, Macro Avg F1-score 0.89
- **Multinomial Naive Bayes**: Accuracy 0.85, Macro Avg F1-score 0.85
- **Linear SVC**: Accuracy 0.88, Macro Avg F1-score 0.88

The **Logistic Regression** model performed the best with an overall accuracy and F1-score of 0.89.


**Reasoning**:
Since Logistic Regression was identified as the best performing model, the next step is to explicitly print its classification report as requested by the subtask.



In [27]:
print("Classification Report for Best Model (Logistic Regression):")
print(classification_report(y_test, y_pred_log_reg))

Classification Report for Best Model (Logistic Regression):
              precision    recall  f1-score   support

    negative       0.90      0.87      0.89      4961
    positive       0.88      0.90      0.89      5039

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



## Final Task

### Subtask:
Summarize the findings, including which model performed best and a brief interpretation of its classification report.


## Summary:

### Q&A

**Which model performed best?**
The Logistic Regression model performed the best.

**Brief interpretation of the best model's (Logistic Regression) classification report:**
The Logistic Regression model achieved an overall accuracy of 0.89. For the 'negative' class, it had a precision of 0.90, recall of 0.87, and an f1-score of 0.89. For the 'positive' class, it showed a precision of 0.88, recall of 0.90, and an f1-score of 0.89. This indicates a strong and balanced performance across both sentiment categories, with a good ability to correctly identify both negative and positive sentiments.

### Data Analysis Key Findings

*   The data was successfully split into training (40,000 samples) and testing (10,000 samples) sets, with 80% for training and 20% for testing.
*   **Logistic Regression** achieved the highest accuracy and F1-score of all evaluated models, both at 0.89.
*   **Linear SVC** followed with an accuracy and F1-score of 0.88.
*   **Multinomial Naive Bayes** showed the lowest performance among the three, with an accuracy and F1-score of 0.85.
*   The classification report for the best-performing Logistic Regression model shows strong performance:
    *   **Accuracy:** 0.89
    *   **Precision (negative):** 0.90
    *   **Recall (negative):** 0.87
    *   **F1-score (negative):** 0.89
    *   **Precision (positive):** 0.88
    *   **Recall (positive):** 0.90
    *   **F1-score (positive):** 0.89

### Insights or Next Steps

*   Consider hyperparameter tuning for the Logistic Regression model to potentially further improve its performance, as it demonstrated the strongest baseline.
*   Explore additional advanced models or ensemble methods (e.g., Random Forest, Gradient Boosting) to see if higher accuracy can be achieved for sentiment classification.
